In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Weather-archive-Jena-V3'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

                  Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  \
0       01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90   
1       01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28   
2       01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31   
3       01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07   
4       01.01.2009 00:50:00    996.51     -8.27    265.15        -9.04   
...                     ...       ...       ...       ...          ...   
420546  31.12.2016 23:20:00   1000.07     -4.05    269.10        -8.13   
420547  31.12.2016 23:30:00    999.93     -3.35    269.81        -8.06   
420548  31.12.2016 23:40:00    999.82     -3.16    270.01        -8.21   
420549  31.12.2016 23:50:00    999.81     -4.23    268.94        -8.53   
420550  01.01.2017 00:00:00    999.82     -4.82    268.36        -8.42   

        rh (%)  VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  \
0        93.30          3.33          3.

In [4]:
data = pd.read_csv('./data.csv')[:5000]

In [5]:
data

                Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0     01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90    93.3   
1     01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28    93.4   
2     01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31    93.9   
3     01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07    94.2   
4     01.01.2009 00:50:00    996.51     -8.27    265.15        -9.04    94.1   
...                   ...       ...       ...       ...          ...     ...   
4995  04.02.2009 16:40:00    971.19      2.16    277.63        -0.30    83.7   
4996  04.02.2009 16:50:00    971.19      1.93    277.39        -0.29    85.2   
4997  04.02.2009 17:00:00    971.22      1.93    277.39        -0.21    85.7   
4998  04.02.2009 17:10:00    971.40      1.87    277.33        -0.24    85.8   
4999  04.02.2009 17:20:00    971.65      1.99    277.42        -0.13    85.8   

      VPmax (mbar)  VPact (mbar)  VPdef

In [6]:
data = data['rho (g/m**3)']

In [7]:
data

0       1307.75
1       1309.80
2       1310.24
3       1309.19
4       1309.00
         ...   
4995    1225.98
4996    1227.03
4997    1227.04
4998    1227.52
4999    1227.31
Name: rho (g/m**3), Length: 5000, dtype: float64

In [8]:
data = data.tolist()

In [9]:
data

[1307.75,
 1309.8,
 1310.24,
 1309.19,
 1309.0,
 1307.86,
 1305.68,
 1305.69,
 1307.17,
 1309.85,
 1311.64,
 1312.25,
 1312.11,
 1312.15,
 1311.37,
 1311.38,
 1311.64,
 1312.18,
 1312.37,
 1313.01,
 1313.15,
 1312.78,
 1313.39,
 1313.61,
 1314.62,
 1316.02,
 1316.16,
 1315.47,
 1316.25,
 1317.19,
 1317.35,
 1317.4,
 1318.68,
 1318.81,
 1316.77,
 1317.71,
 1317.44,
 1315.26,
 1314.23,
 1315.09,
 1317.18,
 1315.98,
 1313.87,
 1313.17,
 1312.67,
 1312.25,
 1311.63,
 1311.65,
 1311.98,
 1311.66,
 1311.26,
 1310.21,
 1309.54,
 1310.14,
 1310.51,
 1309.78,
 1308.43,
 1307.9,
 1307.46,
 1307.76,
 1307.91,
 1308.67,
 1309.41,
 1309.96,
 1310.07,
 1310.43,
 1310.71,
 1310.1,
 1309.08,
 1308.69,
 1307.86,
 1306.98,
 1306.33,
 1305.97,
 1305.44,
 1304.79,
 1303.28,
 1301.73,
 1301.38,
 1301.0,
 1301.76,
 1301.94,
 1301.37,
 1301.67,
 1301.17,
 1300.77,
 1300.69,
 1300.77,
 1300.61,
 1300.51,
 1299.83,
 1300.14,
 1299.4,
 1298.82,
 1298.95,
 1299.17,
 1298.81,
 1298.69,
 1298.96,
 1298.87,
 1299.0

In [10]:
data = torch.from_numpy(np.array(data)).to(device).float().view(1,-1)

In [11]:
data_input = data[:,:-1]
data_target = data[:,1:]

In [12]:
data_input,data_target

(tensor([[1307.7500, 1309.8000, 1310.2400,  ..., 1227.0300, 1227.0400,
          1227.5200]], device='cuda:0'),
 tensor([[1309.8000, 1310.2400, 1309.1899,  ..., 1227.0400, 1227.5200,
          1227.3101]], device='cuda:0'))

In [13]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.output = Linear(self.hidden,1)
    
    def forward(self,X,future):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [14]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)

In [15]:
epochs = 100

In [16]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 128
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.output = Linear(self.hidden,1)
    
    def forward(self,X,future):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [17]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)

In [18]:
epochs = 100

In [19]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        n = data_input.shape[1]
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_input.cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

In [20]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 128
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.output = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            pred = self.output(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [21]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)

In [22]:
epochs = 100

In [23]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        n = data_input.shape[1]
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_input.cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()